# Assignment Data Structure and Solution
#### We can use this Jupyter Notebook to analysis the data structure of different file and variables to help us to solve these three model construction and analysis.

# First Class(CommandLine)
<big>**Calss Function**</big>:
   1. Judge the command line to accord with the stationary Formate.
   2. Print Help Inforamtion if the command formate is wrong.
   3. Default:<br>
      1)'index_nostoplist_nostemming.txt' / queries_nostoplist_nostemming.txt'. <br>
      2) For the First dataset(index?), it's the key vocabularly in the text. <br>
      3) For the second dataset(queries?), it's the summary of all the search queris. <br>
       
**Example Command Formate**:
```
> python ir_engine.py -o result.txt
```

In [1]:
#==============================================================================
# Importing
import sys, getopt, re
import time
#==============================================================================
# Command line processing
class CommandLine:
    def __init__(self):
        opts, args = getopt.getopt(sys.argv[1:], 'hspw:o:')
        opts = dict(opts)
        self.exit = True

        if '-h' in opts:
            self.printHelp()
            return

        if len(args) > 0:
            print("*** ERROR: no arg files - only options! ***", file=sys.stderr)
            self.printHelp()
            return

        if '-w' in opts:
            if opts['-w'] in ('binary', 'tf', 'tfidf'):
                self.termWeighting = opts['-w']
            else:
                warning = (
                    "*** ERROR: term weighting label (opt: -w LABEL)! ***\n"
                    "    -- value (%s) not recognised!\n"
                    "    -- must be one of: binary / tf / tfidf"
                    )  % (opts['-w'])
                print(warning, file=sys.stderr)
                self.printHelp()
                return
        else:
            self.termWeighting = 'binary'

        if '-o' in opts:
            self.outfile = opts['-o']
        else:
            print("*** ERROR: must specify output file (opt: -o FILE) ***",
                  file=sys.stderr)
            self.printHelp()
            return

        if '-s' in opts and '-p' in opts:
            self.indexFile   = 'index_withstoplist_withstemming.txt'
            self.queriesFile = 'queries_withstoplist_withstemming.txt'
        elif '-s' in opts:
            self.indexFile   = 'index_withstoplist_nostemming.txt'
            self.queriesFile = 'queries_withstoplist_nostemming.txt'
        elif '-p' in opts:
            self.indexFile   = 'index_nostoplist_withstemming.txt'
            self.queriesFile = 'queries_nostoplist_withstemming.txt'
        else:
            self.indexFile   = 'index_nostoplist_nostemming.txt'
            self.queriesFile = 'queries_nostoplist_nostemming.txt'
            
        self.exit = False

    def printHelp(self):
        progname = sys.argv[0]
        progname = progname.split('/')[-1] # strip off extended path
        help = __doc__.replace('<PROGNAME>', progname, 1)
        print(help, file=sys.stderr)


# Second Class(IndexLoader)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 dimensions.

<big>**Data Structure(Python Dictionary)**</big>:<br>
   1. {(term name)->{(term name)-> Count}}

In [2]:
#==============================================================================
# Load (precomputed) Index File for (preprocessed) Document Collection
class IndexLoader:
    def __init__(self, indexFile):
        self.index = {}
        docidCountRE = re.compile('(\d+):(\d+)')
        f = open(indexFile, 'r')
        for line in f:
            term = line.split(' ', 1)[0]
            self.index[term] = {}
            for (docid, count) in docidCountRE.findall(line):
                docid = int(docid)
                self.index[term][docid] = int(count)
    def getIndex(self):
        return self.index

# Third Class(Queries)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 or more dimensions.
   3. This class is to extract the queries and get the same dataset formate(python dictionary) as IndexLoader

<big>**Data Structure(Dictionary)**</big>:<br>
   1. {(qid name)->{(qid name)-> count_num}}

In [3]:
class Queries:
    def __init__(self, queriesFile):
        self.qStore = {}
        termCountRE = re.compile('(\w+):(\d+)')
        f = open(queriesFile, 'r')
        for line in f:
            qid = int(line.split(' ', 1)[0])
            self.qStore[qid] = {}
            for (term, count) in termCountRE.findall(line):
                self.qStore[qid][term] = int(count)
    
    def getQuery(self, qid):
        if qid in self.qStore:
            return self.qStore[qid]
        else:
            print("*** ERROR: unknown query identifier (\"%s\") ***" % qid, file=sys.stderr)
            if type(qid) == type(''):
                print('WARNING: query identifiers should be of type: integer', file=sys.stderr)
                print('         -- your query identifier is of type: string', file=sys.stderr)
            print(' -- program exiting', file=sys.stderr)
    
    def qids(self):
        return sorted(self.qStore)

# Forth Class(ResultStore)
<big>**Calss Function**</big>:<br>
<big>**Data Structure(Dictionary)**</big>:<br>

In [4]:
class ResultStore:
    def __init__(self, outfile):
        self.outfile = outfile
        self.results = []

    def store(self, qid, docids):
        if len(docids) > 10:
            docids = docids[:10]
        self.results.append((qid, docids))

    def output(self):
        with open(self.outfile, 'w') as out:
            for (qid, docids) in self.results:
                for docid in docids:
                    print(qid, docid, file=out)

# Target Class(Retrieve)

In [10]:
# TODO: Finish the assignment part
class Retrieve:
    # Create new Retrieve object storing index and termWeighting scheme
    def __init__(self,index, termWeighting):
        self.index = index
        self.termWeighting = termWeighting

    # Method performing retrieval for specified query
    def forQuery(self, query):
        return range(1,11)

<big>**Below Code is the Formate of Command(Please customize with on the basis of your requirements)**</big><br><br>

**How to generate command line As you wish**<br>

`sys.argv` is the python reserved variables and applied to `ArgumentParser.parse_args(args=None, namespace=None)` to be the one of the parameters. This is a list of strings, each of which is obviously derived from command line arguments, separated by spaces. We can redefine this list by below command.

We should delete the key parameters like `python` and put the remains into the list.

In [5]:
# Example Command Formate:
#  > python ir_engine.py -o result.txt
sys.argv = ['ir_engine.py', '-o', 'result.txt']

In [6]:
config = CommandLine()
index = IndexLoader(config.indexFile).getIndex()